In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import time
from datasets import load_dataset
from PIL import Image
from torchvision import transforms
from typing import Dict, Tuple
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00


In [3]:
final_train_df = pd.read_csv("/kaggle/input/vqa-filtered/textpreprocessed_labelencoding.csv")

In [4]:
final_train_df

,image_id,question_id,question_preprocessed,question_type,answer_preprocessed,answers,answer_type,answer_encoded
0,/kaggle/input/visual-question-answering/train2...,262146000,what color is the snow,what color is the,white,"['white', 'white', 'white', 'white', 'white', ...",other,959
1,/kaggle/input/visual-question-answering/train2...,262146001,what is the person doing,what is the person,skiing,"['skiing', 'skiing', 'skiing', 'skiing', 'skii...",other,791
2,/kaggle/input/visual-question-answering/train2...,262146002,what color is the persons headwear,what color is the,red,"['red', 'red', 'red & white', 'black', 'red', ...",other,708
3,/kaggle/input/visual-question-answering/train2...,393221000,is the sky blue,is the,yes,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",yes/no,995
4,/kaggle/input/visual-question-answering/train2...,393221001,is there snow on the mountains,is there,yes,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",yes/no,995
...,...,...,...,...,...,...,...,...
96886,/kaggle/input/visual-question-answering/train2...,393195000,why is she smiling,why,happy,"['heard joke', 'happy', ""she's enjoying her bo...",other,419
96887,/kaggle/input/visual-question-answering/train2...,393195001,is the woman wearing a hat,is the woman,yes,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",yes/no,995
96888,/kaggle/input/visual-question-answering/train2...,393195002,is the woman wearing glasses,is the woman,yes,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",yes/no,995
96889,/kaggle/input/visual-question-answering/train2...,262136000,can he be sharpening blades,none of the above,yes,"['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...",yes/no,995


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import BertTokenizer, AutoImageProcessor

class VQADataset(Dataset):
    def __init__(self, dataframe, bert_tokenizer, image_processor):
        self.dataframe = dataframe
        self.bert_tokenizer = bert_tokenizer
        self.image_processor = image_processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        
        # Process the text
        text_inputs = self.bert_tokenizer(
            row['question_preprocessed'],
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        # Process the image
        image = Image.open(row['image_id']).convert("RGB")
        image_inputs = self.image_processor(images=image, return_tensors='pt')
        
        # Prepare the label
        label = torch.tensor(row['answer_encoded'], dtype=torch.long)
        
        # Merge the inputs
        inputs = {
            'input_ids': text_inputs['input_ids'].squeeze(0),
            'attention_mask': text_inputs['attention_mask'].squeeze(0),
            'pixel_values': image_inputs['pixel_values'].squeeze(0),
            'labels': label
        }
        return inputs


2024-05-17 09:18:31.263482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-17 09:18:31.263583: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-17 09:18:31.402706: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
train_df, test_df = train_test_split(final_train_df, test_size=0.2, random_state=42)

In [7]:
print(len(train_df), len(test_df))

77512 19379


In [8]:
train_df.iloc[0]

image_id                 /kaggle/input/visual-question-answering/train2...
question_id                                                      161751002
question_preprocessed             is the train traveling through grassland
question_type                                                       is the
answer_preprocessed                                                    yes
answers                  ['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'ye...
answer_type                                                         yes/no
answer_encoded                                                         995
Name: 40853, dtype: object

In [9]:
# Initialize tokenizers
bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')
image_processor = AutoImageProcessor.from_pretrained('google/vit-base-patch16-224')

# Create a dataset instance
train_dataset = VQADataset(train_df, bert_tokenizer, image_processor)
eval_dataset = VQADataset(test_df, bert_tokenizer, image_processor)

# Create a DataLoader instance
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(eval_dataset, batch_size=2, shuffle=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

In [10]:
train_dataset[0]

{'input_ids': tensor([  101,  2003,  1996,  3345,  7118,  2083, 20331,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [11]:
from peft import LoraConfig, TaskType
from transformers import BertModel, ViTModel

bert_lora_config = LoraConfig(
    r=8,
    target_modules=["query", "value"],
    lora_alpha=32,
    lora_dropout=0.05
)

vit_lora_config = LoraConfig(
    r=8,
    target_modules=["query", "value"],
    lora_alpha=32,
    lora_dropout=0.05
)

In [12]:
bertLora = BertModel.from_pretrained('google-bert/bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [13]:
vitLora = ViTModel.from_pretrained('google/vit-base-patch16-224')

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from peft import get_peft_model

bert_lora_model = get_peft_model(bertLora, bert_lora_config)
bert_lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 109,777,152 || trainable%: 0.2686


In [15]:
vit_lora_model = get_peft_model(vitLora, vit_lora_config)
vit_lora_model.print_trainable_parameters()

trainable params: 294,912 || all params: 86,684,160 || trainable%: 0.3402


In [16]:
import torch.nn as nn
from peft import get_peft_model
from peft import LoraConfig, TaskType

class VQAModel(nn.Module):
    def __init__(self, bert_model, vit_model, num_labels=1000, intermediate_dim=128):
        super(VQAModel, self).__init__()
        self.bert = bert_model
#         self.bert = bert_lora_model
        self.vit = vit_model
#         self.vit = vit_lora_model
        self.fusion = nn.Sequential(
            nn.Linear(self.bert.config.hidden_size + self.vit.config.hidden_size, intermediate_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
        )
        
        self.classifier = nn.Linear(intermediate_dim, num_labels)
        # self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, pixel_values, labels=None):
        # Get BERT and ViT outputs
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        vit_outputs = self.vit(pixel_values=pixel_values)
        
        fused_output = self.fusion(
            torch.cat(
                [
                    bert_outputs.pooler_output,
                    vit_outputs.pooler_output,
                ],
                dim=1
            )
        )
        
        logits = self.classifier(fused_output)
        
        # Element-wise dot product of BERT and ViT embeddings
#         combined = bert_outputs.last_hidden_state[:, 0, :] * vit_outputs.pooler_output
        
        # Classification
#         logits = self.classifier(combined)
        
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return loss, logits
        else:
            return logits


In [17]:
def compute_metrics(eval_tuple: Tuple[np.ndarray, np.ndarray]) -> Dict[str, float]:
    logits, labels = eval_tuple
    preds = logits.argmax(axis=-1)
    return {
        "acc": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average='weighted'),
        "precision": precision_score(labels, preds, average='weighted'),
        "recall": recall_score(labels, preds, average='weighted'),
    }

In [18]:
# print([(n, type(m)) for n, m in VQAModel().named_modules()])

In [19]:
# result = [n for n, m in VQAModel().named_modules() if isinstance(m, torch.nn.modules.linear.Linear)]
# print(result)
# result.pop()

In [20]:
# from peft import LoraConfig

# config = LoraConfig(
#     target_modules=result,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     r=16,
#     bias="none",
#     modules_to_save=['classifier'],
# )


In [21]:
model = VQAModel(bert_lora_model, vit_lora_model)
model.to(device)

VQAModel(
  (bert): PeftModel(
    (base_model): LoraModel(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=F

In [22]:
from transformers import Trainer, TrainingArguments

# Define the training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working',
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/kaggle/working',
    save_steps=19378,
    save_total_limit=2,
    logging_steps=19378,
    report_to=[],  # Disable logging to wandb
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # assuming you have a validation dataset
    compute_metrics=compute_metrics,  # You can define metrics if needed
)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
# Track time taken for training
start_time = time.time()
train_result = trainer.train()
end_time = time.time()

train_time = end_time - start_time
print(f"Training time: {train_time} seconds")

# # Save the final model
# trainer.save_model('/kaggle/working/vqa_model')

# # Save the training arguments
# training_args.save('/kaggle/working/vqa_model/training_args.bin')

# for epoch in range(training_args.num_train_epochs):
#     print(f"Epoch {epoch}, Loss: {compute_loss(model, train_dataset)}")

Step,Training Loss
19378,3.300000
38756,2.829100
58134,2.650400
77512,2.605600


Training time: 12814.540585279465 seconds


In [24]:
eval_result = trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print(f"Evaluation Metrics: {eval_result}")

Evaluation Metrics: {'eval_loss': 2.578815460205078, 'eval_acc': 0.33902678156767635, 'eval_f1': 0.26997803235866663, 'eval_precision': 0.2699023855687098, 'eval_recall': 0.33902678156767635, 'eval_runtime': 786.063, 'eval_samples_per_second': 24.653, 'eval_steps_per_second': 12.327, 'epoch': 2.0}


In [26]:
print(f"Training Metrics: {train_result.metrics}")

Training Metrics: {'train_runtime': 12814.1936, 'train_samples_per_second': 12.098, 'train_steps_per_second': 6.049, 'total_flos': 0.0, 'train_loss': 2.846283377896326, 'epoch': 2.0}
